In [1]:
import logging
logging.basicConfig(level=logging.DEBUG)

import graphcat.notebook
import numpy

def squared(graph, name, inputs, extent):
    return numpy.power(inputs.getone(None, extent), 2)

graph = graphcat.StreamingGraph()
logger = graphcat.Logger(graph)
graph.set_task("A", graphcat.array(numpy.arange(8)))
graph.set_task("B", squared)
graph.set_links("A", "B")

graphcat.notebook.display(graph)

In [2]:
graph.output("B")

INFO:graphcat.common:Task B updating.
INFO:graphcat.common:Task B executing. Inputs: {None} Extent: None
INFO:graphcat.common:Task A updating.
INFO:graphcat.common:Task A executing. Inputs: {} Extent: None
INFO:graphcat.common:Task A finished. Output: [0 1 2 3 4 5 6 7]
INFO:graphcat.common:Task B finished. Output: [ 0  1  4  9 16 25 36 49]


array([ 0,  1,  4,  9, 16, 25, 36, 49])

That looks pretty familiar - when requested, task "A" returns the array we used to initialize it, and task "B" computes the square of each value from "A".  We can see some new output in the logs, but otherwise everything is working the way it would with static and dynamic graphs.

What if the array was much larger, or task "B" was something truly time consuming, so that we wanted to perform an operation on just a subset? This is what streaming graphs were meant for. To do so, they allow us to specify "extents", which define the subset of a task's output that we want:

In [3]:
graph.output("B", extent=graphcat.ArrayExtent[0:4])

INFO:graphcat.common:Task B updating.
INFO:graphcat.common:Task B executing. Inputs: {None} Extent: slice(0, 4, None)
INFO:graphcat.common:Task A updating.
INFO:graphcat.common:Task A executing. Inputs: {} Extent: slice(0, 4, None)
INFO:graphcat.common:Task A finished. Output: [0 1 2 3]
INFO:graphcat.common:Task B finished. Output: [0 1 4 9]


array([0, 1, 4, 9])

In [4]:
graph.output("A", extent=graphcat.ArrayExtent[0:4])

INFO:graphcat.common:Task A updating.


array([0, 1, 2, 3])

... we get the same result, but the tasks haven't been executed again.  If we use a different extent:

In [5]:
graph.output("A", extent=None)

INFO:graphcat.common:Task A updating.
INFO:graphcat.common:Task A executing. Inputs: {} Extent: None
INFO:graphcat.common:Task A finished. Output: [0 1 2 3 4 5 6 7]


array([0, 1, 2, 3, 4, 5, 6, 7])